In [1]:
import geojson
import numpy as np
from skimage import io
from skimage.draw import polygon
import cv2
from PIL import Image
import os
from shutil import copyfile
import fnmatch

D:\Python\Anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
src_path="E:\data\data_2023_11_3\data_miles"
data_save_path="./data_11_9/"

In [3]:
files=list(set([item.split(".")[0] for item in os.listdir(src_path)]))
tif_files=[item+".ome.tif" for item in files]
geojson_files=[item+".geojson" for item in files]
#17 files 

In [4]:
def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            if any(isinstance(subitem, list) for subitem in item):
                flat_list.extend(flatten_list(item))
            else:
                flat_list.append(item)
        else:
            flat_list.append(item)
    return flat_list
def save_overlap(background_path,mask_image_path,alpha=0.5):
    background_image = cv2.imread(background_path)  # Replace with your background image
    mask_image = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)  # Replace with your mask image
    if mask_image.shape != background_image.shape[:2]:
        mask_image = cv2.resize(mask_image, (background_image.shape[1], background_image.shape[0]))

    # Convert the single-channel mask to a 3-channel mask (to match the background)
    mask_3_channel = cv2.cvtColor(mask_image, cv2.COLOR_GRAY2BGR)

    # Define the transparency weight
    alpha = 0.6

    # Overlay the mask on the background using cv2.addWeighted()
    overlayed_image = cv2.addWeighted(background_image, 1 - alpha, mask_3_channel, alpha, 0)
    return overlayed_image

# Create_labels

In [5]:
for idx,item in enumerate(files):
    curr_path=os.path.join(data_save_path,files[idx])
    os.makedirs(curr_path, exist_ok=True)
    
    file_tif_path=os.path.join(src_path,tif_files[idx])
    file_geojson_path=os.path.join(src_path,geojson_files[idx])
    
    tgt_tif_path=os.path.join(curr_path,tif_files[idx])
    tgt_geojson_path=os.path.join(curr_path,geojson_files[idx])
    
    copyfile(file_tif_path, tgt_tif_path)
    copyfile(file_geojson_path, tgt_geojson_path)
    


    file_path = tgt_tif_path
    annotation_geojson_path = tgt_geojson_path

    annotation_geojson_file = open(annotation_geojson_path)
    collection = geojson.load(annotation_geojson_file)
    try:
        if "features" in collection.keys():
            collection = collection["features"]
        elif "geometries" in collection.keys():
            collection = collection["geometries"]
    except AttributeError:
        # already a list?
        pass

    image = io.imread(file_path)
    binary_image = np.zeros((image.shape[1], image.shape[2]), dtype=np.uint8)
    for i in range(len(collection)):
        if len(collection[i].geometry.coordinates)==1:
            polygon_numpy = np.array(collection[i].geometry.coordinates, dtype=object)
            polygon_numpy2d = polygon_numpy[0, :, :]
            rr, cc = polygon(polygon_numpy2d[:, 0]-1, polygon_numpy2d[:, 1]-1)
        #     binary_image[cc.min():cc.max(),rr.min():rr.max()] = 255
            binary_image[cc,rr] = 255
        else:
            polygon_numpy1=flatten_list(collection[i].geometry.coordinates)
            polygon_numpy2=np.array(polygon_numpy1)
            rr, cc = polygon(polygon_numpy2[:, 0]-1, polygon_numpy2[:, 1]-1)
        #     binary_image[cc.min():cc.max(),rr.min():rr.max()] = 255
            binary_image[cc,rr] = 255
    cv2.imwrite(os.path.join(curr_path,files[idx]+ '_seg.png'), binary_image)
    

In [12]:
patch_size=(512,512)
center_size=80
average_brightness_score=30
std_area_of_interest=1
total_count=0

In [13]:
for folder in os.listdir(data_save_path):
    curr_path=os.path.join(data_save_path,folder)
    folder_path=curr_path

    save_patch_dir = folder_path + "/patch/"
    os.makedirs(save_patch_dir, exist_ok=True)

    save_seg_dir = folder_path + "/seg/"
    os.makedirs(save_seg_dir, exist_ok=True)


    file_path=[item for item in os.listdir(folder_path) if item.endswith("tif")][0]
    seg_path=[item for item in os.listdir(folder_path) if item.endswith("png")][0]
    original_image = Image.open(os.path.join(curr_path,file_path))
    seg_image = Image.open(os.path.join(curr_path,seg_path))
    image_cv = cv2.imread(os.path.join(curr_path,file_path))
    width, height = original_image.width, original_image.height

    # grid = [[0] * (width // patch_size[0]) for _ in range(height // patch_size[1])]
    saved_patches = []
    print("------{}-patch------".format(folder))
    for y in range(0, original_image.height, patch_size[1]):
        for x in range(0, original_image.width, patch_size[0]):
            patch = original_image.crop((x, y, x + patch_size[0], y + patch_size[1]))
            seg = seg_image.crop((x, y, x + patch_size[0], y + patch_size[1]))

            patch_array = np.array(patch)
            area_of_interest = patch_array[center_size:-center_size, center_size:-center_size]
            # Check starting pixel value
            average_brightness = np.mean(area_of_interest)
            std_dev = np.std(area_of_interest)
            # Check std deviation threshold

            if x + patch_size[0] <= original_image.width \
                    and y + patch_size[1] <= original_image.height \
                    and std_dev >= std_area_of_interest:

                if patch.getpixel((0, 0)) == (255, 255, 255) \
                        or patch.getpixel((0, 511)) == (255, 255, 255) \
                        or patch.getpixel((511, 0)) == (255, 255, 255) \
                        or patch.getpixel((511, 511)) == (255, 255, 255):
                    if average_brightness <= average_brightness_score:
                        if std_dev >= std_area_of_interest:
                            saved_patches.append(((y, x), patch))
                            # Save the patchb
                            patch_filename = os.path.join(save_patch_dir, f"patch_x_{x}_{x + 512}_y_{y}_{y + 512}.png")
                            patch.save(patch_filename)
                            out_p=patch_filename.split("/")[-1]
                            print(f"Saved patch {out_p}")
                            # Save the seg
                            seg_filename = os.path.join(save_seg_dir, f"seg_x_{x}_{x + 512}_y_{y}_{y + 512}.png")
                            seg.save(seg_filename)
                            out_s=seg_filename.split("/")[-1]
                            print(f"Saved seg {out_s}")


                else:
                    saved_patches.append(((y, x), patch))
                    # Save the patchb
                    patch_filename = os.path.join(save_patch_dir, f"patch_x_{x}_{x + 512}_y_{y}_{y + 512}.png")
                    patch.save(patch_filename)
                    out_p=patch_filename.split("/")[-1]
                    print(f"Saved patch {out_p}")
                    # Save the seg
                    seg_filename = os.path.join(save_seg_dir, f"seg_x_{x}_{x + 512}_y_{y}_{y + 512}.png")
                    seg.save(seg_filename)
                    out_s=seg_filename.split("/")[-1]
                    print(f"Saved seg {out_s}")
    print("------{}-patch finised : {}------".format(folder,len(saved_patches)))
    print("------{}-heatmap------".format(folder))
    overlay = image_cv.copy()
    for (y, x), patch in saved_patches:
        cv2.rectangle(overlay, (x, y), (x + patch_size[0], y + patch_size[1]), (0, 255, 0),
                      -1)  # Draw a green rectangle
    # Apply the overlay to the original image
    output = cv2.addWeighted(image_cv, 0.8, overlay, 0.2, 0)
    cv2.imwrite(folder_path + '/image_new_{}_patches.jpg'.format(len(saved_patches)), output)
    total_count+=len(saved_patches)
print("-----total-{}-patches------".format(total_count))

------Mx_BEGIN TMA EDGE_A15-patch------
Saved patch patch_x_1024_1536_y_0_512.png
Saved seg seg_x_1024_1536_y_0_512.png
Saved patch patch_x_1536_2048_y_0_512.png
Saved seg seg_x_1536_2048_y_0_512.png
Saved patch patch_x_2048_2560_y_0_512.png
Saved seg seg_x_2048_2560_y_0_512.png
Saved patch patch_x_2560_3072_y_0_512.png
Saved seg seg_x_2560_3072_y_0_512.png
Saved patch patch_x_3072_3584_y_0_512.png
Saved seg seg_x_3072_3584_y_0_512.png
Saved patch patch_x_3584_4096_y_0_512.png
Saved seg seg_x_3584_4096_y_0_512.png
Saved patch patch_x_4096_4608_y_0_512.png
Saved seg seg_x_4096_4608_y_0_512.png
Saved patch patch_x_0_512_y_512_1024.png
Saved seg seg_x_0_512_y_512_1024.png
Saved patch patch_x_512_1024_y_512_1024.png
Saved seg seg_x_512_1024_y_512_1024.png
Saved patch patch_x_1024_1536_y_512_1024.png
Saved seg seg_x_1024_1536_y_512_1024.png
Saved patch patch_x_1536_2048_y_512_1024.png
Saved seg seg_x_1536_2048_y_512_1024.png
Saved patch patch_x_2048_2560_y_512_1024.png
Saved seg seg_x_2048_

Saved seg seg_x_512_1024_y_4608_5120.png
Saved patch patch_x_1024_1536_y_4608_5120.png
Saved seg seg_x_1024_1536_y_4608_5120.png
Saved patch patch_x_1536_2048_y_4608_5120.png
Saved seg seg_x_1536_2048_y_4608_5120.png
Saved patch patch_x_2048_2560_y_4608_5120.png
Saved seg seg_x_2048_2560_y_4608_5120.png
Saved patch patch_x_2560_3072_y_4608_5120.png
Saved seg seg_x_2560_3072_y_4608_5120.png
Saved patch patch_x_3072_3584_y_4608_5120.png
Saved seg seg_x_3072_3584_y_4608_5120.png
Saved patch patch_x_3584_4096_y_4608_5120.png
Saved seg seg_x_3584_4096_y_4608_5120.png
Saved patch patch_x_4096_4608_y_4608_5120.png
Saved seg seg_x_4096_4608_y_4608_5120.png
Saved patch patch_x_4608_5120_y_4608_5120.png
Saved seg seg_x_4608_5120_y_4608_5120.png
Saved patch patch_x_5120_5632_y_4608_5120.png
Saved seg seg_x_5120_5632_y_4608_5120.png
Saved patch patch_x_5632_6144_y_4608_5120.png
Saved seg seg_x_5632_6144_y_4608_5120.png
Saved patch patch_x_512_1024_y_5120_5632.png
Saved seg seg_x_512_1024_y_5120_56

Saved patch patch_x_2048_2560_y_3072_3584.png
Saved seg seg_x_2048_2560_y_3072_3584.png
Saved patch patch_x_2560_3072_y_3072_3584.png
Saved seg seg_x_2560_3072_y_3072_3584.png
Saved patch patch_x_3072_3584_y_3072_3584.png
Saved seg seg_x_3072_3584_y_3072_3584.png
Saved patch patch_x_3584_4096_y_3072_3584.png
Saved seg seg_x_3584_4096_y_3072_3584.png
Saved patch patch_x_4096_4608_y_3072_3584.png
Saved seg seg_x_4096_4608_y_3072_3584.png
Saved patch patch_x_4608_5120_y_3072_3584.png
Saved seg seg_x_4608_5120_y_3072_3584.png
Saved patch patch_x_5120_5632_y_3072_3584.png
Saved seg seg_x_5120_5632_y_3072_3584.png
Saved patch patch_x_5632_6144_y_3072_3584.png
Saved seg seg_x_5632_6144_y_3072_3584.png
Saved patch patch_x_512_1024_y_3584_4096.png
Saved seg seg_x_512_1024_y_3584_4096.png
Saved patch patch_x_1024_1536_y_3584_4096.png
Saved seg seg_x_1024_1536_y_3584_4096.png
Saved patch patch_x_1536_2048_y_3584_4096.png
Saved seg seg_x_1536_2048_y_3584_4096.png
Saved patch patch_x_2048_2560_y_35

Saved patch patch_x_512_1024_y_2560_3072.png
Saved seg seg_x_512_1024_y_2560_3072.png
Saved patch patch_x_1536_2048_y_2560_3072.png
Saved seg seg_x_1536_2048_y_2560_3072.png
Saved patch patch_x_2048_2560_y_2560_3072.png
Saved seg seg_x_2048_2560_y_2560_3072.png
Saved patch patch_x_2560_3072_y_2560_3072.png
Saved seg seg_x_2560_3072_y_2560_3072.png
Saved patch patch_x_3072_3584_y_2560_3072.png
Saved seg seg_x_3072_3584_y_2560_3072.png
Saved patch patch_x_5120_5632_y_2560_3072.png
Saved seg seg_x_5120_5632_y_2560_3072.png
Saved patch patch_x_5632_6144_y_2560_3072.png
Saved seg seg_x_5632_6144_y_2560_3072.png
Saved patch patch_x_512_1024_y_3072_3584.png
Saved seg seg_x_512_1024_y_3072_3584.png
Saved patch patch_x_1024_1536_y_3072_3584.png
Saved seg seg_x_1024_1536_y_3072_3584.png
Saved patch patch_x_2048_2560_y_3072_3584.png
Saved seg seg_x_2048_2560_y_3072_3584.png
Saved patch patch_x_2560_3072_y_3072_3584.png
Saved seg seg_x_2560_3072_y_3072_3584.png
Saved patch patch_x_3072_3584_y_3072

Saved patch patch_x_2048_2560_y_3072_3584.png
Saved seg seg_x_2048_2560_y_3072_3584.png
Saved patch patch_x_2560_3072_y_3072_3584.png
Saved seg seg_x_2560_3072_y_3072_3584.png
Saved patch patch_x_3072_3584_y_3072_3584.png
Saved seg seg_x_3072_3584_y_3072_3584.png
Saved patch patch_x_3584_4096_y_3072_3584.png
Saved seg seg_x_3584_4096_y_3072_3584.png
Saved patch patch_x_4096_4608_y_3072_3584.png
Saved seg seg_x_4096_4608_y_3072_3584.png
Saved patch patch_x_4608_5120_y_3072_3584.png
Saved seg seg_x_4608_5120_y_3072_3584.png
Saved patch patch_x_5120_5632_y_3072_3584.png
Saved seg seg_x_5120_5632_y_3072_3584.png
Saved patch patch_x_512_1024_y_3584_4096.png
Saved seg seg_x_512_1024_y_3584_4096.png
Saved patch patch_x_1024_1536_y_3584_4096.png
Saved seg seg_x_1024_1536_y_3584_4096.png
Saved patch patch_x_1536_2048_y_3584_4096.png
Saved seg seg_x_1536_2048_y_3584_4096.png
Saved patch patch_x_2048_2560_y_3584_4096.png
Saved seg seg_x_2048_2560_y_3584_4096.png
Saved patch patch_x_2560_3072_y_35

Saved seg seg_x_4608_5120_y_2560_3072.png
Saved patch patch_x_5120_5632_y_2560_3072.png
Saved seg seg_x_5120_5632_y_2560_3072.png
Saved patch patch_x_5632_6144_y_2560_3072.png
Saved seg seg_x_5632_6144_y_2560_3072.png
Saved patch patch_x_6144_6656_y_2560_3072.png
Saved seg seg_x_6144_6656_y_2560_3072.png
Saved patch patch_x_0_512_y_3072_3584.png
Saved seg seg_x_0_512_y_3072_3584.png
Saved patch patch_x_512_1024_y_3072_3584.png
Saved seg seg_x_512_1024_y_3072_3584.png
Saved patch patch_x_1024_1536_y_3072_3584.png
Saved seg seg_x_1024_1536_y_3072_3584.png
Saved patch patch_x_1536_2048_y_3072_3584.png
Saved seg seg_x_1536_2048_y_3072_3584.png
Saved patch patch_x_2048_2560_y_3072_3584.png
Saved seg seg_x_2048_2560_y_3072_3584.png
Saved patch patch_x_2560_3072_y_3072_3584.png
Saved seg seg_x_2560_3072_y_3072_3584.png
Saved patch patch_x_3072_3584_y_3072_3584.png
Saved seg seg_x_3072_3584_y_3072_3584.png
Saved patch patch_x_3584_4096_y_3072_3584.png
Saved seg seg_x_3584_4096_y_3072_3584.png


Saved patch patch_x_1024_1536_y_2048_2560.png
Saved seg seg_x_1024_1536_y_2048_2560.png
Saved patch patch_x_1536_2048_y_2048_2560.png
Saved seg seg_x_1536_2048_y_2048_2560.png
Saved patch patch_x_2048_2560_y_2048_2560.png
Saved seg seg_x_2048_2560_y_2048_2560.png
Saved patch patch_x_2560_3072_y_2048_2560.png
Saved seg seg_x_2560_3072_y_2048_2560.png
------Mx_SA TMA 2_G07-patch finised : 28------
------Mx_SA TMA 2_G07-heatmap------
------Mx_SA TMA 5_G02-patch------
Saved patch patch_x_1024_1536_y_0_512.png
Saved seg seg_x_1024_1536_y_0_512.png
Saved patch patch_x_1536_2048_y_0_512.png
Saved seg seg_x_1536_2048_y_0_512.png
Saved patch patch_x_2048_2560_y_0_512.png
Saved seg seg_x_2048_2560_y_0_512.png
Saved patch patch_x_2560_3072_y_0_512.png
Saved seg seg_x_2560_3072_y_0_512.png
Saved patch patch_x_512_1024_y_512_1024.png
Saved seg seg_x_512_1024_y_512_1024.png
Saved patch patch_x_1024_1536_y_512_1024.png
Saved seg seg_x_1024_1536_y_512_1024.png
Saved patch patch_x_1536_2048_y_512_1024.

Saved seg seg_x_2048_2560_y_1024_1536.png
Saved patch patch_x_0_512_y_1536_2048.png
Saved seg seg_x_0_512_y_1536_2048.png
Saved patch patch_x_512_1024_y_1536_2048.png
Saved seg seg_x_512_1024_y_1536_2048.png
Saved patch patch_x_1024_1536_y_1536_2048.png
Saved seg seg_x_1024_1536_y_1536_2048.png
Saved patch patch_x_1536_2048_y_1536_2048.png
Saved seg seg_x_1536_2048_y_1536_2048.png
Saved patch patch_x_2048_2560_y_1536_2048.png
Saved seg seg_x_2048_2560_y_1536_2048.png
Saved patch patch_x_0_512_y_2048_2560.png
Saved seg seg_x_0_512_y_2048_2560.png
Saved patch patch_x_512_1024_y_2048_2560.png
Saved seg seg_x_512_1024_y_2048_2560.png
Saved patch patch_x_1024_1536_y_2048_2560.png
Saved seg seg_x_1024_1536_y_2048_2560.png
Saved patch patch_x_1536_2048_y_2048_2560.png
Saved seg seg_x_1536_2048_y_2048_2560.png
Saved patch patch_x_2048_2560_y_2048_2560.png
Saved seg seg_x_2048_2560_y_2048_2560.png
Saved patch patch_x_512_1024_y_2560_3072.png
Saved seg seg_x_512_1024_y_2560_3072.png
Saved patch 

------Mx_Southampton TMA Research Block 5_G10-patch------
Saved patch patch_x_0_512_y_0_512.png
Saved seg seg_x_0_512_y_0_512.png
Saved patch patch_x_512_1024_y_0_512.png
Saved seg seg_x_512_1024_y_0_512.png
Saved patch patch_x_1024_1536_y_0_512.png
Saved seg seg_x_1024_1536_y_0_512.png
Saved patch patch_x_1536_2048_y_0_512.png
Saved seg seg_x_1536_2048_y_0_512.png
Saved patch patch_x_2048_2560_y_0_512.png
Saved seg seg_x_2048_2560_y_0_512.png
Saved patch patch_x_0_512_y_512_1024.png
Saved seg seg_x_0_512_y_512_1024.png
Saved patch patch_x_512_1024_y_512_1024.png
Saved seg seg_x_512_1024_y_512_1024.png
Saved patch patch_x_1024_1536_y_512_1024.png
Saved seg seg_x_1024_1536_y_512_1024.png
Saved patch patch_x_1536_2048_y_512_1024.png
Saved seg seg_x_1536_2048_y_512_1024.png
Saved patch patch_x_0_512_y_1024_1536.png
Saved seg seg_x_0_512_y_1024_1536.png
Saved patch patch_x_512_1024_y_1024_1536.png
Saved seg seg_x_512_1024_y_1024_1536.png
Saved patch patch_x_1024_1536_y_1024_1536.png
Saved 

Saved patch patch_x_2560_3072_y_512_1024.png
Saved seg seg_x_2560_3072_y_512_1024.png
Saved patch patch_x_3072_3584_y_512_1024.png
Saved seg seg_x_3072_3584_y_512_1024.png
Saved patch patch_x_0_512_y_1024_1536.png
Saved seg seg_x_0_512_y_1024_1536.png
Saved patch patch_x_512_1024_y_1024_1536.png
Saved seg seg_x_512_1024_y_1024_1536.png
Saved patch patch_x_1024_1536_y_1024_1536.png
Saved seg seg_x_1024_1536_y_1024_1536.png
Saved patch patch_x_1536_2048_y_1024_1536.png
Saved seg seg_x_1536_2048_y_1024_1536.png
Saved patch patch_x_2048_2560_y_1024_1536.png
Saved seg seg_x_2048_2560_y_1024_1536.png
Saved patch patch_x_2560_3072_y_1024_1536.png
Saved seg seg_x_2560_3072_y_1024_1536.png
Saved patch patch_x_3072_3584_y_1024_1536.png
Saved seg seg_x_3072_3584_y_1024_1536.png
Saved patch patch_x_0_512_y_1536_2048.png
Saved seg seg_x_0_512_y_1536_2048.png
Saved patch patch_x_512_1024_y_1536_2048.png
Saved seg seg_x_512_1024_y_1536_2048.png
Saved patch patch_x_1024_1536_y_1536_2048.png
Saved seg 